## BC_ResNet


In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

In [2]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [3]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tue Jul 12 11:25:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   37C    P0    39W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/ubuntu/data/hmd/murmur/train'
test_folder = '/home/ubuntu/data/hmd/murmur/test'


In [6]:
model_folder = 'ddws1'

## Modeling



In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [8]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

### get feature 함수확장: 음성피쳐 옵션들과, 추가 음성들 고려한 피쳐추출

In [9]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [10]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 256,
                  'win_length':512,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10
}

In [11]:
features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)
features_test, mm_lbs_test, out_lbs_test, _, _, _ = get_features_3lb_all(test_folder, patient_files_test, **params_feature)

melspec:  100 313
cqt:  80 157
stft:  257 313


utils/get_feature.py:599: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  features[k1] = np.array(features[k1])


melspec:  100 313
cqt:  80 157
stft:  257 313


### DDWS


In [36]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.activations import relu, softmax, swish

def SubSpectralNorm(S, A = True, eps=1e-5):
    # S : number of sub-bands
    # A : 'Sub' Transform type if True, 
    #     'All' Transform type if False.          <- cannot be implemented yet.
    # Can be applied only for image inputs.
    
    
    def f(x):
        if S == 1:
            y = BatchNormalization()(x)
            
        else:
            F = x.shape[1]             # number of frequencies of the input.
            if F % S == 0:
                Q = F // S             # length of each sub-band.
                subs = []
                for i in range(S):     
                    subs.append(x[:, i*Q:(i+1)*Q, :,:])
                    
                for i in range(S):
                    subs[i] = BatchNormalization()(subs[i])
                    
            else:
                Q = F // S
                subs = []
                for i in range(S-1):
                    subs.append(x[:,i*Q:(i+1)*Q,:,:])
                    
                subs.append(x[:,(S-1)*Q:,:,:])      # the shape of x and y must be the same.
                
                for i in range(S):
                    subs[i] = BatchNormalization()(subs[i])
                    
            
            y = tf.concat(subs, axis=1)
            
        return y
    
    return f


def TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1),
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.3):
    
    def f(x):
        if out_channels:
            y = Conv2D(filters=out_channels, kernel_size=(1,1), strides=strides,
                       activation=None, kernel_initializer='he_uniform')(x)
            y = BatchNormalization()(y)
            y = relu(y)
            
            z = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
                                activation=None, kernel_initializer='he_uniform')(y)
            z = SubSpectralNorm(S=2)(z)
            
        else :
            z = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
                                activation=None, kernel_initializer='he_uniform')(x)
            z = SubSpectralNorm(S=2)(z)
            
        z = relu(z)
        z = DepthwiseConv2D(kernel_size=temporal_kernel_size, strides=(1,1), padding='same',
                                activation=None, kernel_initializer='he_uniform')(z)
        z = SubSpectralNorm(S=2)(z)
            #        y = relu(y)
        z = swish(z)
            
        if out_channels:
            z = Conv2D(filters=y.shape[3], kernel_size=(1,1), strides=(1,1),
                    activation='relu', kernel_initializer='he_uniform')(z)
        else :
            z = Conv2D(filters=x.shape[3], kernel_size=(1,1), strides=(1,1),
                       activation='relu', kernel_initializer='he_uniform')(z)
                
        z = SpatialDropout2D(dropout_rate)(z)
        ############################
            
        if out_channels:
            return add([y, z])
        else:
            return add([x, z])
            
    return f


    
def BC_ResMax(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
              freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1):

    def f(x):
        if out_channels:
            y = Conv2D(filters=out_channels, kernel_size=(1,1), strides=strides, 
                       activation=None, kernel_initializer='he_uniform')(x)
            y = BatchNormalization()(y)
            y = relu(y)
            
            y = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
                                 activation=None, kernel_initializer='he_uniform')(y)
#            y2 = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
#                                 activation=None, kernel_initializer='he_uniform')(y)
#            y = tf.keras.layers.maximum([y1, y2])

        else :
            y = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
                                 activation=None, kernel_initializer='he_uniform')(x)
#            y2 = DepthwiseConv2D(kernel_size=freq_kernel_size, strides=(1,1), padding='same',
#                                 activation=None, kernel_initializer='he_uniform')(x)
#            y = tf.keras.layers.maximum([y1, y2])
            
        y = SubSpectralNorm(S=2)(y)                   # Should be changed to SSN
        y = relu(y)
        ############################
        
        z = AveragePooling2D(pool_size=(y.shape[1],1))(y)
        
        ########### f1 #############
        z = DepthwiseConv2D(kernel_size=temporal_kernel_size, strides=(1,1), dilation_rate=dilation_rate,
                            padding='same', activation=None, kernel_initializer='he_uniform')(z)
        z = BatchNormalization()(z)
        z = swish(z)

        z = Conv2D(filters=y.shape[3], kernel_size=(1,1), strides=(1,1),
                   activation='relu', kernel_initializer='he_uniform')(z)
        z = SpatialDropout2D(dropout_rate)(z)                  
        ############################
        
        
        ########### BC #############
        z = UpSampling2D(size=(y.shape[1],1), interpolation='nearest')(z)
        ############################
        
        if out_channels:
            return add([y, z])
        else: 
            return add([x, y, z])
        
    return f

def ResMax(n_output, k, l, upscale=False):
    def f(x):
        conv1_1 = Conv2D(filters = n_output, kernel_size =k, strides=(1, 1), padding='same', activation=None)(x)
        conv1_2 = Conv2D(filters = n_output, kernel_size =k, strides=(1, 1), padding='same', activation=None)(x)
        h = maximum([conv1_1, conv1_2])
        if l :
            conv1_1 = Conv2D(filters = n_output, kernel_size =k, strides=(1, 1), padding='same', activation=None)(h)
            conv1_2 = Conv2D(filters = n_output, kernel_size =k, strides=(1, 1), padding='same', activation=None)(h)
            h = maximum([conv1_1, conv1_2])
        if upscale:
            f = Conv2D(kernel_size=1, filters=n_output, strides=1, padding='same')(x)
        else:
            f = x
        return add([f, h])
    return f




def get_DDWS_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(mel1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(mel1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 24, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    mel2 = GlobalAveragePooling2D()(x)
    
    ## cqt embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(cqt1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(cqt1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 24, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    mel2 = GlobalAveragePooling2D()(x)
    cqt2 = GlobalAveragePooling2D()(x)

    ## stft embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(stft1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(stft1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 24, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    stft2 = GlobalAveragePooling2D()(x)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res1 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy','AUC'])
    return(model)

def get_DDWS_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(mel1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(mel1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

#    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
#                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
#    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
#                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
#    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

#    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
#                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
#    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
#                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
#    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    mel2 = GlobalAveragePooling2D()(x)
    
    ## cqt embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(cqt1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(cqt1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 24, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    mel2 = GlobalAveragePooling2D()(x)
    cqt2 = GlobalAveragePooling2D()(x)

    ## stft embedding
    x1 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(stft1)
    x2 = Conv2D(filters = 16, kernel_size = (3,3), activation = None)(stft1)
    x = maximum([x1, x2])
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 24, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 32, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 48, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    x = TF_ResMax_seq(out_channels = 64, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = TF_ResMax_seq(out_channels = False, strides=(1,1), dilation_rate=(1,1), 
                  freq_kernel_size=(3,1), temporal_kernel_size=(1,3), dropout_rate=0.1)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
    stft2 = GlobalAveragePooling2D()(x)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res2 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy','AUC'])
    return(model)

In [37]:
#model1 = get_toy5_1(mel_input_shape, cqt_input_shape, stft_input_shape)
#model2 = get_toy5_2(mel_input_shape, cqt_input_shape, stft_input_shape)


In [38]:
model1 = get_DDWS_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)
model2 = get_DDWS_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)


In [39]:
model1.summary()

Model: "model_384"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cqt (InputLayer)                [(None, 80, 157, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_17295 (Conv2D)           (None, 78, 155, 16)  160         cqt[0][0]                        
__________________________________________________________________________________________________
conv2d_17296 (Conv2D)           (None, 78, 155, 16)  160         cqt[0][0]                        
__________________________________________________________________________________________________
maximum_1153 (Maximum)          (None, 78, 155, 16)  0           conv2d_17295[0][0]               
                                                                 conv2d_17296[0][0]       

In [40]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 13s 197ms/step - loss: 0.7163 - accuracy: 0.7592 - auc: 0.8355 - val_loss: 0.6463 - val_accuracy: 0.7448 - val_auc: 0.8205
Epoch 2/100
40/40 [==============================] - 6s 157ms/step - loss: 0.6047 - accuracy: 0.7644 - auc: 0.8652 - val_loss: 0.7297 - val_accuracy: 0.7448 - val_auc: 0.8661
Epoch 3/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6113 - accuracy: 0.7576 - auc: 0.8639 - val_loss: 0.7050 - val_accuracy: 0.7448 - val_auc: 0.8387
Epoch 4/100
40/40 [==============================] - 6s 157ms/step - loss: 0.6133 - accuracy: 0.7565 - auc: 0.8574 - val_loss: 0.7012 - val_accuracy: 0.7448 - val_auc: 0.8316
Epoch 5/100
40/40 [==============================] - 6s 156ms/step - loss: 0.5880 - accuracy: 0.7777 - auc: 0.8639 - val_loss: 0.6801 - val_accuracy: 0.7448 - val_auc: 0.8157
Epoch 6/100
40/40 [==============================] - 6s 159ms/step - loss: 0.6218 - accuracy: 0.7460 - auc: 0.8665 - val_los

40/40 [==============================] - 6s 158ms/step - loss: 0.5844 - accuracy: 0.7560 - auc: 0.8771 - val_loss: 0.5744 - val_accuracy: 0.7861 - val_auc: 0.8838
Epoch 48/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5804 - accuracy: 0.7533 - auc: 0.8753 - val_loss: 0.5670 - val_accuracy: 0.7845 - val_auc: 0.8837
Epoch 49/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5804 - accuracy: 0.7648 - auc: 0.8774 - val_loss: 0.5504 - val_accuracy: 0.7829 - val_auc: 0.8859
Epoch 50/100
40/40 [==============================] - 6s 160ms/step - loss: 0.5759 - accuracy: 0.7670 - auc: 0.8690 - val_loss: 0.5806 - val_accuracy: 0.7623 - val_auc: 0.8788
Epoch 51/100
40/40 [==============================] - 6s 155ms/step - loss: 0.5742 - accuracy: 0.7738 - auc: 0.8807 - val_loss: 0.5595 - val_accuracy: 0.7924 - val_auc: 0.8884
Epoch 52/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5734 - accuracy: 0.7614 - auc: 0.8807 - val_loss: 0.572

Epoch 94/100
40/40 [==============================] - 6s 156ms/step - loss: 0.5857 - accuracy: 0.7532 - auc: 0.8791 - val_loss: 0.5383 - val_accuracy: 0.7956 - val_auc: 0.8852
Epoch 95/100
40/40 [==============================] - 6s 154ms/step - loss: 0.5581 - accuracy: 0.7667 - auc: 0.8825 - val_loss: 0.5376 - val_accuracy: 0.7987 - val_auc: 0.8853
Epoch 96/100
40/40 [==============================] - 6s 156ms/step - loss: 0.5703 - accuracy: 0.7623 - auc: 0.8824 - val_loss: 0.5377 - val_accuracy: 0.7987 - val_auc: 0.8855
Epoch 97/100
40/40 [==============================] - 6s 158ms/step - loss: 0.5893 - accuracy: 0.7579 - auc: 0.8813 - val_loss: 0.5372 - val_accuracy: 0.7971 - val_auc: 0.8867
Epoch 98/100
40/40 [==============================] - 6s 155ms/step - loss: 0.5899 - accuracy: 0.7539 - auc: 0.8770 - val_loss: 0.5374 - val_accuracy: 0.7987 - val_auc: 0.8867
Epoch 99/100
40/40 [==============================] - 6s 164ms/step - loss: 0.5667 - accuracy: 0.7705 - auc: 0.8792 - va

트레이닝 accuracy 와 auc 가 validation accuracy, auc 와 거의 유사함

In [41]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 16s 274ms/step - loss: 0.8414 - accuracy: 0.4985 - auc: 0.4948 - val_loss: 0.8687 - val_accuracy: 0.5008 - val_auc: 0.4129
Epoch 2/100
40/40 [==============================] - 7s 170ms/step - loss: 0.7458 - accuracy: 0.5083 - auc: 0.5044 - val_loss: 0.6909 - val_accuracy: 0.5293 - val_auc: 0.5479
Epoch 3/100
40/40 [==============================] - 7s 171ms/step - loss: 0.7489 - accuracy: 0.4721 - auc: 0.4861 - val_loss: 0.7319 - val_accuracy: 0.4992 - val_auc: 0.5665
Epoch 4/100
40/40 [==============================] - 6s 160ms/step - loss: 0.7182 - accuracy: 0.5093 - auc: 0.5419 - val_loss: 0.7464 - val_accuracy: 0.4992 - val_auc: 0.4850
Epoch 5/100
40/40 [==============================] - 7s 166ms/step - loss: 0.7269 - accuracy: 0.5227 - auc: 0.5064 - val_loss: 0.7990 - val_accuracy: 0.4992 - val_auc: 0.4370
Epoch 6/100
40/40 [==============================] - 7s 171ms/step - loss: 0.7298 - accuracy: 0.5089 - auc: 0.4982 - val_los

40/40 [==============================] - 6s 161ms/step - loss: 0.6890 - accuracy: 0.5509 - auc: 0.5674 - val_loss: 0.6468 - val_accuracy: 0.6513 - val_auc: 0.7061
Epoch 48/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6870 - accuracy: 0.5443 - auc: 0.5880 - val_loss: 0.6424 - val_accuracy: 0.6783 - val_auc: 0.7195
Epoch 49/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6891 - accuracy: 0.5305 - auc: 0.5797 - val_loss: 0.6486 - val_accuracy: 0.6593 - val_auc: 0.7025
Epoch 50/100
40/40 [==============================] - 6s 165ms/step - loss: 0.6889 - accuracy: 0.5494 - auc: 0.5730 - val_loss: 0.6504 - val_accuracy: 0.6466 - val_auc: 0.6983
Epoch 51/100
40/40 [==============================] - 6s 165ms/step - loss: 0.6926 - accuracy: 0.5262 - auc: 0.5661 - val_loss: 0.6493 - val_accuracy: 0.6466 - val_auc: 0.6988
Epoch 52/100
40/40 [==============================] - 6s 161ms/step - loss: 0.6940 - accuracy: 0.5359 - auc: 0.5598 - val_loss: 0.647

Epoch 94/100
40/40 [==============================] - 6s 160ms/step - loss: 0.6906 - accuracy: 0.5321 - auc: 0.5667 - val_loss: 0.6504 - val_accuracy: 0.6498 - val_auc: 0.7084
Epoch 95/100
40/40 [==============================] - 6s 160ms/step - loss: 0.6904 - accuracy: 0.5401 - auc: 0.5752 - val_loss: 0.6493 - val_accuracy: 0.6513 - val_auc: 0.7122
Epoch 96/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6852 - accuracy: 0.5606 - auc: 0.5674 - val_loss: 0.6500 - val_accuracy: 0.6450 - val_auc: 0.7099
Epoch 97/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6904 - accuracy: 0.5247 - auc: 0.5647 - val_loss: 0.6500 - val_accuracy: 0.6498 - val_auc: 0.7105
Epoch 98/100
40/40 [==============================] - 7s 166ms/step - loss: 0.6918 - accuracy: 0.5405 - auc: 0.5607 - val_loss: 0.6498 - val_accuracy: 0.6450 - val_auc: 0.7103
Epoch 99/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6880 - accuracy: 0.5611 - auc: 0.5722 - va

### 팀코드 수정해야 할 함수들..

In [42]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'ddws1',
 'model2': 'ddws2',
 'model_fnm1': 'ddws1/ddws1_model1.hdf5',
 'model_fnm2': 'ddws1/ddws2_model2.hdf5'}

In [43]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [44]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [45]:
model_folder

'ddws1'

In [46]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'ddws1', m_name2 = 'ddws2', param_feature = params_feature)

1

In [47]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [48]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'ddws1',
 'model2': 'ddws2',
 'model_fnm1': 'ddws1/ddws1_model1.hdf5',
 'model_fnm2': 'ddws1/ddws2_model2.hdf5'}

In [49]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy1' :
        model1 = get_toy5_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'] )
    if model['model2'] == 'toy2' :
        model2 = get_toy5_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'])
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'lcnn2' :
        model2 = get_LCNN_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model1'] == 'resmax1' :
        model1 = get_ResMax_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'resmax2' :
        model2 = get_ResMax_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model1'] == 'ddws1' :
        model1 = get_DDWS_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'ddws2' :
        model2 = get_DDWS_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model1'] == 'bcresmax1' :
        model1 = get_BCResMax_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'bcresmax2' :
        model2 = get_BCResMax_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

### 대회 평가용 run_model 함수 (수정 불필요)

In [50]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [51]:
## 우리 모형 저장된 폴더이름
model_folder

'ddws1'

In [52]:
## 테스트 데이터 폴더
test_folder

'/home/ubuntu/data/hmd/murmur/test'

In [53]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/home/ubuntu/hmd/notebooks/out_ddws1'

In [54]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...


Done.


In [55]:
!cat /home/ubuntu/hmd/notebooks/out_ddws1/49628.csv

#49628
Present,Unknown,Absent,Abnormal,Normal
0,0,1,0,1
0.16892476,0.010964039,0.8201113,0.4091206,0.59087944


In [56]:
from evaluate_model import *

In [57]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.628,0.390,0.281,0.728,0.375,25689.450

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.553,0.569,0.327,0.487,0.160,25689.450

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.634,0.674,0.577
AUPRC,0.299,0.135,0.736
F-measure,0.000,0.000,0.842
Accuracy,0.000,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.553,0.553
AUPRC,0.619,0.518
F-measure,0.000,0.655
Accuracy,0.000,1.000



### Threshold 변경해가며 결과 탐색

In [58]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.1643587870680628
0.8252893641361255
0.4018099971727749
0.5981900063350785


In [59]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.628,0.390,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.553,0.569,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.634,0.674,0.577
AUPRC,0.299,0.135,0.736
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.553,0.553
AUPRC,0.619,0.518
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.628,0.390,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.553,0.569,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.634,0.674,0.577
AUPRC,0.299,0.135,0.736
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU